# Import libraries

In [1]:
import os, glob, csv, sys, re
from os import path
import pandas as pd
from bs4 import BeautifulSoup

# Define directories

In [2]:
userhome = os.path.expanduser('~')
analyze_dir = userhome + r'/mining-forum/analyze/'
summ_dir = userhome + r'/mining-forum/analyze/csv_files/summarize/'

# Open cleaned-file dataset

In [3]:
data = pd.read_csv(summ_dir + 'all_joined_files.csv')
data

,file_number,message_id,title,author
0,10,message #23,Some thoughts,Chris Gross
1,100,message #255,Webex: Target Management Meeting,Doug Gaff
2,1000,message #2645,Re: GMF errors,Richard Gronback
3,10000,message #30746,[BUILD] R1.0-I,Alex Panchenko
4,10001,message #30751,Stumped by installation.,Eclipse User
5,10002,message #30757,Object Creation monitor Caller,Eclipse User
6,10003,message #30761,Problems with too many clicks on buttons,Roberto Sanchez
7,10004,message #30762,How to get a classloader for the project in a ...,Eclipse User
8,10005,message #30763,Can't get update manager to function correctly,Eclipse User
9,10006,message #30770,New STEM Design Document Wiki,Daniel Ford


In [4]:
print ("Number of files:", len(data))

Number of files: 289066


In [5]:
data.iloc[0][0]

10

Define function to determine the directory

In [6]:
def dir_num(x):
    if (x < 44000):
        y = 12
    elif (x < 88000):
        y = 13
    elif (x < 132000):
        y = 14
    elif (x < 176000):
        y = 15
    elif (x < 220000):
        y = 16
    elif (x < 440000):
        y = 17
    elif (x < 660000):
        y = 18
    elif (x < 880000):
        y = 19
    else:
        y = 20
    
    return y

# Extracting topics and users category

In [7]:
info = []
for r in range(0,len(data)):
    fnum = data.iloc[r][0]
    files = userhome + r'/mining-forum/data/forumfiles_' + str(dir_num(fnum)) + '/' + str(fnum)
    
    sys.stdout.write('\rExtracting file: %i' %(r+1) + '/%i' %(len(data)) + ' --> %s' %(files))
    sys.stdout.flush()
    
    f = open(files)
    f = f.read()
    
    uname = []
    href = []
    author_id = []
    user_role = []
    
    file_num = re.search('\/(\d+)', files)
    file_num = file_num.group(1)
    
    try:
        soup = BeautifulSoup(f, 'html.parser')
        topic = soup.find('a', attrs={'class':'MsgSubText'})
        for title in topic.stripped_strings:
            title = title

        msid = []
        soup = BeautifulSoup(f, 'html.parser')
        for message_id in soup.find_all('td', attrs={'class':'MsgR1 vt al MsgSubText'}):
            for mid in message_id.find_all('span', attrs={'class':'SmallText'}):
                mi = []
                for msgid in mid.find_all('a'):
                    mi.append(msgid.text)
                msid.append(mi)

        message_id = []
        for m in range(0,len(msid)):
            if len(msid[m]) > 1:
                mess = msid[m][0] + " is a reply to " + msid[m][1]
            else:
                mess = msid[m][0]
            message_id.append(mess)
        
        topic = []
        soup = BeautifulSoup(f, 'html.parser')
        top = soup.find('td', attrs={'class':'ForumBackground'})
        for tops in top.find_all('a'):
            topic.append(tops.text)
        topic = topic[8:10]

        soup = BeautifulSoup(f, 'html.parser')
        for table in soup.find_all('table', attrs={'class':'ContentTable'}):
            for userlink in table.find_all('td', attrs={'class':'MsgSpacer'}):
                for table2 in userlink.find_all('table', attrs={'class':'ContentTable'}):
                    for user in table2.find_all('td', attrs={'class':'msgud'}):
                        for usr in user.find_all('a', href=True):
                            href.append(usr['href']) 
                            for hr, ref in enumerate(href):
                                try:
                                    aut_id = re.search('(\d+)', ref)
                                    aut_id = aut_id.group()
                                except:
                                    aut_id = 'n/a'                      

                            username = usr.text
                            if username == '':
                                username = 'Eclipse User'

                            uname.append(username)
                            author_id.append(aut_id)

                            break;

                    for role in table2.find_all('div', attrs={'class':'ctags'}):
                        urole = role.text
                        user_role.append(urole)

        x = 0
        for n, aid in enumerate(author_id):
            if aid == 'n/a':
                temp = [file_num, topic[0], topic[1], title, message_id[n], uname[n], href[n].replace("/donate","n/a"), aid, 'n/a']
            else:
                temp = [file_num, topic[0], topic[1], title, message_id[n], uname[n], href[n].replace("/donate","n/a"), aid, user_role[x]]
                x = x + 1
            info.append(temp)
            
    except:
        pass
    
print ("\nFiltering files are finished")

Extracting file: 289066/289066 --> /Users/YusufNugroho/mining-forum/data/forumfiles_20/9999972
Filtering files are finished


In [8]:
info

[['10',
  'Eclipse Projects',
  'BIRT',
  'Some thoughts',
  'message #23',
  'Chris Gross',
  'index.php?t=usrinfo&id=8093&',
  '8093',
  'Senior Member'],
 ['10',
  'Eclipse Projects',
  'BIRT',
  'Some thoughts',
  'message #24 is a reply to message #23',
  'Jody Garnett',
  'index.php?t=usrinfo&id=12960&',
  '12960',
  'Junior Member'],
 ['10',
  'Eclipse Projects',
  'BIRT',
  'Some thoughts',
  'message #25 is a reply to message #23',
  'Scott Rosenbaum',
  'index.php?t=usrinfo&id=14908&',
  '14908',
  'Senior Member'],
 ['10',
  'Eclipse Projects',
  'BIRT',
  'Some thoughts',
  'message #26 is a reply to message #25',
  'Chris Gross',
  'index.php?t=usrinfo&id=8093&',
  '8093',
  'Senior Member'],
 ['100',
  'Eclipse Projects',
  'DSDP - Target Management',
  'Webex: Target Management Meeting',
  'message #255',
  'Doug Gaff',
  'index.php?t=usrinfo&id=13750&',
  '13750',
  'Senior Member'],
 ['1000',
  'Modeling',
  'GMF (Graphical Modeling Framework)',
  'Re: GMF errors',
  '

In [9]:
with open (summ_dir + 'list_of_thread_categories_and_users.csv', 'w') as csvfile:
    writers = csv.writer(csvfile, delimiter = ",")
    header = ['file_number', 'main_topic', 'subtopic', 'title', 'message_id', 'author_name', 'author_link', 'author_id', 'author_role']
    writers.writerow(header)
    for item in info:
        writers.writerow(item)
print ('CSV file has been created')

CSV file has been created


In [10]:
df_threadcat = pd.read_csv(summ_dir + 'list_of_thread_categories_and_users.csv')
df_threadcat

,file_number,main_topic,subtopic,title,message_id,author_name,author_link,author_id,author_role
0,10,Eclipse Projects,BIRT,Some thoughts,message #23,Chris Gross,index.php?t=usrinfo&id=8093&,8093,Senior Member
1,10,Eclipse Projects,BIRT,Some thoughts,message #24 is a reply to message #23,Jody Garnett,index.php?t=usrinfo&id=12960&,12960,Junior Member
2,10,Eclipse Projects,BIRT,Some thoughts,message #25 is a reply to message #23,Scott Rosenbaum,index.php?t=usrinfo&id=14908&,14908,Senior Member
3,10,Eclipse Projects,BIRT,Some thoughts,message #26 is a reply to message #25,Chris Gross,index.php?t=usrinfo&id=8093&,8093,Senior Member
4,100,Eclipse Projects,DSDP - Target Management,Webex: Target Management Meeting,message #255,Doug Gaff,index.php?t=usrinfo&id=13750&,13750,Senior Member
5,1000,Modeling,GMF (Graphical Modeling Framework),Re: GMF errors,message #2645,Richard Gronback,index.php?t=usrinfo&id=17086&,17086,Senior Member
6,1000,Modeling,GMF (Graphical Modeling Framework),Re: GMF errors,message #4085 is a reply to message #2645,Chris Lenz,index.php?t=usrinfo&id=27804&,27804,Senior Member
7,10000,Eclipse Projects,Dynamic Languages Toolkit (DLTK),[BUILD] R1.0-I,message #30746,Alex Panchenko,index.php?t=usrinfo&id=47448&,47448,Senior Member
8,10001,Newcomers,Newcomers,Stumped by installation.,message #30751,Eclipse User,n/a,n/a,n/a
9,10001,Newcomers,Newcomers,Stumped by installation.,message #30788 is a reply to message #30751,Eclipse User,n/a,n/a,n/a
